In [1]:
import os
import math
import torchvision 
import torch.nn as nn
import torch
import cv2
from tqdm.notebook import tqdm
import numpy as np # linear algebra
import pandas as pd
import gc

In [2]:
import pandas as pd
train_df = pd.read_csv('data/imaterialist-fashion-2020-fgvc7/train.csv')

In [3]:
attr = train_df.AttributesIds
clss = train_df.ClassId
not_exist = attr.isna()
non_nan_classes=[]
for i in range(len(not_exist)):
    if not not_exist[i]:
        if clss[i] not in non_nan_classes:
            non_nan_classes.append(clss[i])
            
non_nan_classes.sort()

In [7]:
filtered_df = pd.DataFrame(train_df[train_df.ClassId.isin(non_nan_classes)])

In [8]:
class CONFIG():
    LABELS_LENGTH = 294

In [9]:
'''class AttrModel(nn.Module):
    def __init__(self, num_classes=294):
        super(AttrModel, self).__init__()
        self.num_classes = num_classes
        Base_Model = torchvision.models.resnet18(pretrained=True)
        
        children = list(Base_Model.children())
        
        self.conv = children[0]
        self.batch_norm = children[1]
        self.relu = children[2]
        self.maxpool = children[3]
        self.blocks1 = children[4]
        self.blocks2 = children[5]
        self.blocks3 = children[6]
        self.blocks4 = children[7]
        self.adp_avg_pool = children[8]
        self.fc = nn.Linear(children[9].in_features, self.num_classes)
        
        
    def forward(self, x):
        x = self.conv(x)
        x = self.batch_norm(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.blocks1(x)
        x = self.blocks2(x)
        x = self.blocks3(x)
        x = self.blocks4(x)
        x = self.adp_avg_pool(x)
        x = self.fc(x)
        x = torch.nn.functional.sigmoid(x)
        return x'''

'class AttrModel(nn.Module):\n    def __init__(self, num_classes=294):\n        super(AttrModel, self).__init__()\n        self.num_classes = num_classes\n        Base_Model = torchvision.models.resnet18(pretrained=True)\n        \n        children = list(Base_Model.children())\n        \n        self.conv = children[0]\n        self.batch_norm = children[1]\n        self.relu = children[2]\n        self.maxpool = children[3]\n        self.blocks1 = children[4]\n        self.blocks2 = children[5]\n        self.blocks3 = children[6]\n        self.blocks4 = children[7]\n        self.adp_avg_pool = children[8]\n        self.fc = nn.Linear(children[9].in_features, self.num_classes)\n        \n        \n    def forward(self, x):\n        x = self.conv(x)\n        x = self.batch_norm(x)\n        x = self.relu(x)\n        x = self.maxpool(x)\n        x = self.blocks1(x)\n        x = self.blocks2(x)\n        x = self.blocks3(x)\n        x = self.blocks4(x)\n        x = self.adp_avg_pool(x)\n  

In [10]:
class Attr_Dataset(torch.utils.data.Dataset):
    def __init__(self, config,  df):
        self.IMGS = list(filtered_df.ImageId)
        self.config = config
        self.df = df
        
        self.Enc = list(self.df.EncodedPixels)
        self.H = list(self.df.Height)
        self.W = list(self.df.Width)
        self.Attr = list(self.df.AttributesIds)
        
    def vectorize_labels(self, labels):
        vector = np.array([0]*self.config.LABELS_LENGTH)
        for idx in labels:
            vector[idx]=1
        return vector
            
    def change_label_indices(self, labels):
        for i in range(len(labels)):
            if labels[i]>234:
                labels[i]-=47
        return labels
    
    def make_single_mask(self, encoded_string, height, width):
        splitted_string = np.array(list(map(int, encoded_string.split()))).reshape(-1,2)
        mask = np.zeros((height*width), dtype=np.uint8)
        for start_indice, run_length in splitted_string:
            start_indice-=1
            mask[start_indice:start_indice+run_length] = 1
        return mask.reshape((height, width), order='F')
        
        
    def __getitem__(self, idx):
        img = np.array(cv2.cvtColor(cv2.imread('data/imaterialist-fashion-2020-fgvc7/train/' + self.IMGS[idx] + '.jpg'), cv2.COLOR_BGR2RGB)/255, dtype=np.float32)
        mask = self.make_single_mask(self.Enc[idx], self.H[idx], self.W[idx])
        mask_ = np.stack((mask,mask,mask), axis=-1)
        img = img*mask_
        img = cv2.resize(img, (1024, 1024), interpolation = cv2.INTER_AREA)
        labels = self.Attr[idx]
        if not (labels!=labels):        
            labels = self.change_label_indices(list(map(int, labels.split(','))))
        else:
            labels=[]
        label_vector = self.vectorize_labels(labels)
        
        
        return np.moveaxis(img, -1, 0), label_vector
    
    def __len__(self):
        return len(self.IMGS)

def collate_fn(batch):
    return tuple(zip(*batch))

In [11]:
class AttrModel(nn.Module):
    def __init__(self, num_classes=294):
        super(AttrModel, self).__init__()
        self.num_classes = num_classes
        Base_Model = torchvision.models.resnet18(pretrained=True, progress=True)
        Base_Model.fc = nn.Linear(Base_Model.fc.in_features, num_classes)
        self.Model_Backend = Base_Model
        
        self.sigmoid = torch.nn.Sigmoid()
    def forward(self, x):
        x = self.Model_Backend(x)
        x = self.sigmoid(x)
        return x

In [12]:
saving_steps = 200
epoch=0
batch_size=12
last = 672

In [13]:
#AttrM = AttrModel()
AttrM = torch.load('models/Base.pt')   
conf = CONFIG()
Attr_dataset = Attr_Dataset(conf, filtered_df)
Attr_data_loader = torch.utils.data.DataLoader(Attr_dataset, batch_size=8, shuffle=True)

In [14]:
criterion = nn.BCELoss()
params = [p for p in AttrM.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,momentum=0.9, weight_decay=0.0005)
DEVICE = torch.device('cuda:1')
_ = AttrM.to(DEVICE)
_ = AttrM.train()

In [15]:

loss_tracking=0
for i in tqdm(range(last, len(Attr_dataset)//batch_size, 1)):
    images = []
    targets = []
    
    for j in range(batch_size):
        I, T = Attr_dataset[i*batch_size+j]
        images.append(I)
        targets.append(T)
        
    images = torch.tensor(images, dtype=torch.float32).to(DEVICE)
    targets = torch.tensor(targets, dtype=torch.float32).to(DEVICE)
    
    out = AttrM(images)
    loss =  criterion(out, targets)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print('loss at step ', i, 'is', loss.item())
    loss_tracking+=loss.item()
    
    if i%saving_steps==0:
        print('================================  Model Saved  ================================')
        torch.save(AttrM, 'models/Attr_epoch_'+str(epoch)+'_step_'+str(i)+'_Loss_'+str(loss_tracking/(i+1-last)))
        torch.cuda.empty_cache()
        gc.collect()


loss at step  672 is 0.05351759493350983
loss at step  673 is 0.05278264358639717
loss at step  674 is 0.0606030598282814
loss at step  675 is 0.048182904720306396
loss at step  676 is 0.06026555597782135
loss at step  677 is 0.04453951492905617
loss at step  678 is 0.0445040799677372
loss at step  679 is 0.0501612164080143
loss at step  680 is 0.050962407141923904
loss at step  681 is 0.056361038237810135
loss at step  682 is 0.05171194300055504
loss at step  683 is 0.049835916608572006
loss at step  684 is 0.054386772215366364
loss at step  685 is 0.05372697114944458
loss at step  686 is 0.050018180161714554
loss at step  687 is 0.061788804829120636
loss at step  688 is 0.04037660360336304
loss at step  689 is 0.056798990815877914
loss at step  690 is 0.04131759703159332
loss at step  691 is 0.05348939076066017
loss at step  692 is 0.04956731200218201
loss at step  693 is 0.04834343120455742
loss at step  694 is 0.04926472529768944
loss at step  695 is 0.056187793612480164
loss at st

C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:402: UserWarning: Couldn't retrieve source code for container of type AttrModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


loss at step  801 is 0.04177973419427872
loss at step  802 is 0.05586016923189163
loss at step  803 is 0.046533066779375076
loss at step  804 is 0.04777514562010765
loss at step  805 is 0.04107840731739998
loss at step  806 is 0.049701716750860214
loss at step  807 is 0.045169346034526825
loss at step  808 is 0.05805554240942001
loss at step  809 is 0.04891274496912956
loss at step  810 is 0.049430664628744125
loss at step  811 is 0.05674364045262337
loss at step  812 is 0.046965692192316055
loss at step  813 is 0.048289086669683456
loss at step  814 is 0.055626947432756424
loss at step  815 is 0.0428868867456913
loss at step  816 is 0.054217997938394547
loss at step  817 is 0.04882197082042694
loss at step  818 is 0.04798629879951477
loss at step  819 is 0.04907384142279625
loss at step  820 is 0.048190098255872726
loss at step  821 is 0.051931459456682205
loss at step  822 is 0.04793308675289154
loss at step  823 is 0.04350828379392624
loss at step  824 is 0.048675935715436935
loss a

loss at step  999 is 0.04601070657372475
loss at step  1000 is 0.03405667841434479
================================  Model Saved  ================================
loss at step  1001 is 0.04190324619412422
loss at step  1002 is 0.042517293244600296
loss at step  1003 is 0.03898550570011139
loss at step  1004 is 0.04385773092508316
loss at step  1005 is 0.051316164433956146
loss at step  1006 is 0.048057921230793
loss at step  1007 is 0.03624714910984039
loss at step  1008 is 0.047682877629995346
loss at step  1009 is 0.03340322524309158
loss at step  1010 is 0.029842033982276917
loss at step  1011 is 0.043463047593832016
loss at step  1012 is 0.050587091594934464
loss at step  1013 is 0.0379394069314003
loss at step  1014 is 0.056424304842948914
loss at step  1015 is 0.047186266630887985
loss at step  1016 is 0.046380914747714996
loss at step  1017 is 0.05592458322644234
loss at step  1018 is 0.040074706077575684
loss at step  1019 is 0.04412051662802696
loss at step  1020 is 0.05162411

loss at step  1191 is 0.047684572637081146
loss at step  1192 is 0.04299648851156235
loss at step  1193 is 0.039015479385852814
loss at step  1194 is 0.033576373010873795
loss at step  1195 is 0.03345489874482155
loss at step  1196 is 0.03840284422039986
loss at step  1197 is 0.04689905047416687
loss at step  1198 is 0.04500482603907585
loss at step  1199 is 0.03474261611700058
loss at step  1200 is 0.03868996724486351
================================  Model Saved  ================================
loss at step  1201 is 0.030370168387889862
loss at step  1202 is 0.042273107916116714
loss at step  1203 is 0.04371834173798561
loss at step  1204 is 0.04721643030643463
loss at step  1205 is 0.04417109861969948
loss at step  1206 is 0.0516110397875309
loss at step  1207 is 0.043276384472846985
loss at step  1208 is 0.05104317516088486
loss at step  1209 is 0.03353770822286606
loss at step  1210 is 0.036999572068452835
loss at step  1211 is 0.04062625393271446
loss at step  1212 is 0.04475057

loss at step  1383 is 0.04638289660215378
loss at step  1384 is 0.05171908810734749
loss at step  1385 is 0.04847464710474014
loss at step  1386 is 0.04676055163145065
loss at step  1387 is 0.04543260857462883
loss at step  1388 is 0.051447395235300064
loss at step  1389 is 0.05356360971927643
loss at step  1390 is 0.04443008080124855
loss at step  1391 is 0.03717942535877228
loss at step  1392 is 0.05052400380373001
loss at step  1393 is 0.033467575907707214
loss at step  1394 is 0.03681168332695961
loss at step  1395 is 0.03575574606657028
loss at step  1396 is 0.04490463435649872
loss at step  1397 is 0.047654591500759125
loss at step  1398 is 0.04160338267683983
loss at step  1399 is 0.05367637425661087
loss at step  1400 is 0.029620494693517685
================================  Model Saved  ================================
loss at step  1401 is 0.04151009023189545
loss at step  1402 is 0.0397162064909935
loss at step  1403 is 0.05211733281612396
loss at step  1404 is 0.03195254132

loss at step  1575 is 0.04900915548205376
loss at step  1576 is 0.06433957070112228
loss at step  1577 is 0.041744545102119446
loss at step  1578 is 0.03738892823457718
loss at step  1579 is 0.05592961981892586
loss at step  1580 is 0.0392436683177948
loss at step  1581 is 0.04062354937195778
loss at step  1582 is 0.05849076807498932
loss at step  1583 is 0.034357666969299316
loss at step  1584 is 0.041222844272851944
loss at step  1585 is 0.04265617951750755
loss at step  1586 is 0.03664451837539673
loss at step  1587 is 0.04260089620947838
loss at step  1588 is 0.047414299100637436
loss at step  1589 is 0.0327291265130043
loss at step  1590 is 0.050153013318777084
loss at step  1591 is 0.03186272084712982
loss at step  1592 is 0.0379069559276104
loss at step  1593 is 0.05115680769085884
loss at step  1594 is 0.03925231844186783
loss at step  1595 is 0.046080850064754486
loss at step  1596 is 0.04256683215498924
loss at step  1597 is 0.04261303320527077
loss at step  1598 is 0.0413542

loss at step  1767 is 0.056806404143571854
loss at step  1768 is 0.03995731845498085
loss at step  1769 is 0.04695530980825424
loss at step  1770 is 0.040680959820747375
loss at step  1771 is 0.048600658774375916
loss at step  1772 is 0.046994730830192566
loss at step  1773 is 0.03438325598835945
loss at step  1774 is 0.0392194464802742
loss at step  1775 is 0.036006540060043335
loss at step  1776 is 0.046612806618213654
loss at step  1777 is 0.03743681684136391
loss at step  1778 is 0.04243630915880203
loss at step  1779 is 0.04405377805233002
loss at step  1780 is 0.050116684287786484
loss at step  1781 is 0.0371754951775074
loss at step  1782 is 0.051873937249183655
loss at step  1783 is 0.03923952579498291
loss at step  1784 is 0.03902656212449074
loss at step  1785 is 0.042500875890254974
loss at step  1786 is 0.04995395988225937
loss at step  1787 is 0.04147876054048538
loss at step  1788 is 0.05121228098869324
loss at step  1789 is 0.042410410940647125
loss at step  1790 is 0.03

loss at step  1958 is 0.05055830255150795
loss at step  1959 is 0.05001654103398323
loss at step  1960 is 0.03987341746687889
loss at step  1961 is 0.041048359125852585
loss at step  1962 is 0.05088772624731064
loss at step  1963 is 0.03404058516025543
loss at step  1964 is 0.03852245584130287
loss at step  1965 is 0.041525520384311676
loss at step  1966 is 0.04532378539443016
loss at step  1967 is 0.043445050716400146
loss at step  1968 is 0.04864737391471863
loss at step  1969 is 0.04815904423594475
loss at step  1970 is 0.037218671292066574
loss at step  1971 is 0.0450030080974102
loss at step  1972 is 0.05292849242687225
loss at step  1973 is 0.032135725021362305
loss at step  1974 is 0.03740433603525162
loss at step  1975 is 0.03477131575345993
loss at step  1976 is 0.042451873421669006
loss at step  1977 is 0.03981240466237068
loss at step  1978 is 0.04797176644206047
loss at step  1979 is 0.024135489016771317
loss at step  1980 is 0.04150734841823578
loss at step  1981 is 0.0501

KeyboardInterrupt: 

In [ ]:
o